# Introducing OpenManipulator-X
Wir werden hier die Grundlegenden Funktionalitäten des OpenManipulator-X Roboterarms kennenlernen.

## Installation
Perform the following code just once. It will download the necessary files and compile them. This will take a while.
Don'T forget to select julia as the kernel for this notebook. It should be in ```/snap/bin/julia```

In [ ]:
import Pkg
Pkg.generate("OpenMEnv") # generate a new package
Pkg.activate("OpenMEnv") # activate the package
Pkg.add("Revise")
Pkg.add("PyCall")
Pkg.add("RobotOS")

## Environment Initialization
If you performed the lines ahead you can just skip this step and activate the virtual environment "OpenMEnv". To use Methods implemented in OpenManipulatorBase.jl we include the file and import the module OpenManipulatorBase. The module OpenManipulatorBase contains all the necessary functions to control the OpenManipulator-X. 

In [ ]:
import Pkg
Pkg.activate("OpenMEnv") 
using Revise
include("OpenManipulatorBase.jl")
#def OpenManipulatorBase
using .OpenManipulatorBase

## Start the Simulation 
Next we start Gazebo. Gazebo is a 3D simulator for robots. It is used to simulate the OpenManipulator-X. The following code will start Gazebo and load the OpenManipulator-X model:

In [ ]:
OpenManipulatorBase.start_gazebo() 

Now before we start the conroller we have to start the simulation in gazebo. This can be done by pressing the play button in the bottom left corner of the gazebo window. 

![Image](./gazebo.png)

## Start the Controller
Next we intitialize the OpenManipulator-X controller. It starts several Services that can be used to control the Robot. Information about the robot can also be obtained via topics where the controller publishes.
You can get an overview of Topics and Services on the following website: https://emanual.robotis.com/docs/en/platform/openmanipulator_x/ros_controller_msg/

In [ ]:
OpenManipulatorBase.start_controller(true)

In [ ]:
OpenManipulatorBase.init()

If the function above does not finish: Did you press play in Gazebo? 

## Control the Robot
The following code will move the robot to a predefined position. The position is defined by the joint angles of the robot. The joint angles are defined in radians. Since we can move 4 joints we need to define 4 angles. The following code will move the robot to the position [0.6, 0.6, -0.5, -0.1]. The positions are from the bottom of the robot to the top, hence [ID11, ID12, ID13, ID14]. Hence the first position is the angle of the base joint, the second the angle of the shoulder joint, the third the angle of the elbow joint and the last the angle of the wrist joint. The ID15 is for the Gripper and is not used here.

![Image](./OpenManipulator.png)

In [ ]:
OpenManipulatorBase.set_joint_angles([0.6, 0.6, -0.5, -0.1])

The following code will move the robot to the position [0.0,0.0,0.0,0.0]. This is the initial position of the robot.

In [ ]:
OpenManipulatorBase.reset_robot()

We can also control the robot together with a parameter that defines how long the Robot will take for the path. The following code will move the robot to different positions with each in 1.0 second and waits by performing nothing for 1.0 second. 

In [ ]:
joint_rad_positions = [[-1.6, -0.6, 0.5, -0.1],
                        [-0.0, -0.0, 0.0, -0.0],
                        [1.6, 0.6, -0.5, -0.1],
                        [-0.6, -0.6, -0.5, 0.2]]
length = size(joint_rad_positions)[1]
path_time = 1.0
for i in 1:10
    for i in 1:length
        OpenManipulatorBase.send_goal_joint_space_path(joint_rad_positions[i], path_time)
        OpenManipulatorBase.rossleep(1.0)
    end
end

Ok, so we can control the robot. But we can also get information about the robot. The following code will print the current position of the robot. The position is given in the same order as the position we used to control the robot. 

In [ ]:
subscribe_to_joint_states()

In [ ]:
OpenManipulatorBase.set_joint_angles([0.6, 0.6, -0.5, -0.1])


We can also control the gripper with the following command:

In [ ]:
send_goal_tool_control(-0.01)

# Motivation zur Kinematik
The Controller defines a coordinate system (https://emanual.robotis.com/docs/en/platform/openmanipulator_x/specification/#hardware-specification) and with that we are actually able to control the robot by defining the position of the end effector. The following code will move the robot to the position [0., 0.2, 0.2] in the coordinate system of the robot.

In [ ]:
send_goal_task_space_path_position_only([0., 0.2, 0.2], 1.0)

In [ ]:
send_goal_task_space_path_position_only([0.2, 0., 0.2], 1.0)

In [ ]:
send_goal_task_space_path_position_only([0.2, 0.2, 0.04], 1.0)

Now you can try to move the robot to different positions. You can also try to move the robot to a position that is not reachable. What happens? 